In [ ]:
GITHUB_USERNAME = "$GITHUB_USERNAME$"
GITHUB_REF = "$GITHUB_REF$"
NOTEBOOK_TYPE = "$NOTEBOOK_TYPE$"
PYTHON_VERSION = "$PYTHON_VERSION$"
IPYTHON_VERSION = "$IPYTHON_VERSION$"

In [ ]:
import warnings
from pathlib import Path

import requests


warnings.filterwarnings('error', module='davos')

if NOTEBOOK_TYPE == 'colab':
    # utils module doesn't exist on colab VM, so get current version from GitHub
    utils_module = Path('utils.py').resolve()
    response = requests.get(f'https://raw.githubusercontent.com/{GITHUB_USERNAME}/davos/{GITHUB_REF}/tests/utils.py')
    utils_module.write_text(response.text)
    # also need to install davos locally
    from utils import install_davos
    install_davos(source='github', ref=GITHUB_REF, fork=GITHUB_USERNAME)

In [ ]:
from pprint import pformat
from textwrap import dedent

import davos
from davos.core.regexps import pip_installed_pkgs_regex, smuggle_statement_regex

from utils import run_tests

In [ ]:
IPYTHON_SHELL = get_ipython()

# tests for `davos.core.regexps`

In [ ]:
def test_pip_installed_regex_single():
    """
    should match a single package name when no dependencies needed to be 
    installed. stdout used for test comes from running
    '!pip install git+https://github.com/ContextLab/davos.git' in a 
    Colab notebook.
    """
    stdout = dedent("""\
    Collecting git+https://github.com/ContextLab/davos.git
      Cloning https://github.com/ContextLab/davos.git to /tmp/pip-req-build-4272zzqa
      Running command git clone -q https://github.com/ContextLab/davos.git /tmp/pip-req-build-4272zzqa
      Installing build dependencies ... done
      Getting requirements to build wheel ... done
        Preparing wheel metadata ... done
    Building wheels for collected packages: davos
      Building wheel for davos (PEP 517) ... done
      Created wheel for davos: filename=davos-0.0.1-cp37-none-any.whl size=37733 sha256=d02890824aa9e9ae1b0c7e801113c4120a43f4e9ba7220db3227d46411c7b45c
      Stored in directory: /tmp/pip-ephem-wheel-cache-y28fng42/wheels/62/30/bc/79958ce75e105bdcf95c737091e62372429850960d6628e277
    Successfully built davos
    Installing collected packages: davos
    Successfully installed davos-0.0.1""")
    expected = ['davos-0.0.1']
    result = pip_installed_pkgs_regex.findall(stdout)
    assert result == expected, (
        f"Expected: {expected}\nFound: {result}"
    )

In [ ]:
def test_pip_installed_regex_multi():
    """
    should match names of installed package and all just-installed 
    dependencies when when any dependencies needed to be installed. 
    stdout used for test comes from running '!pip install hypertools' in 
    a Colab notebook.
    """
    stdout = dedent("""\
    Collecting hypertools
      Downloading https://files.pythonhosted.org/packages/09/8d/54736bd23e346fcf33fa15d63c2403c05a191a0c1047ed01a24b68794fc6/hypertools-0.7.0-py3-none-any.whl (59kB)
         |████████████████████████████████| 61kB 2.5MB/s 
    Requirement already satisfied: matplotlib>=1.5.1 in /usr/local/lib/python3.7/dist-packages (from hypertools) (3.2.2)
    Requirement already satisfied: future in /usr/local/lib/python3.7/dist-packages (from hypertools) (0.16.0)
    Collecting deepdish
      Downloading https://files.pythonhosted.org/packages/6e/39/2a47c852651982bc5eb39212ac110284dd20126bdc7b49bde401a0139f5d/deepdish-0.3.6-py2.py3-none-any.whl
    Requirement already satisfied: numpy>=1.10.4 in /usr/local/lib/python3.7/dist-packages (from hypertools) (1.19.5)
    Requirement already satisfied: scikit-learn!=0.22,<0.24,>=0.19.1 in /usr/local/lib/python3.7/dist-packages (from hypertools) (0.22.2.post1)
    Requirement already satisfied: requests in /usr/local/lib/python3.7/dist-packages (from hypertools) (2.23.0)
    Requirement already satisfied: six in /usr/local/lib/python3.7/dist-packages (from hypertools) (1.15.0)
    Requirement already satisfied: seaborn>=0.8.1 in /usr/local/lib/python3.7/dist-packages (from hypertools) (0.11.1)
    Requirement already satisfied: scipy>=1.0.0 in /usr/local/lib/python3.7/dist-packages (from hypertools) (1.4.1)
    Collecting PPCA>=0.0.2
      Downloading https://files.pythonhosted.org/packages/16/7f/7195bf3742e19076a21a9c3250a4f11c87153bb4ea3dcaf1077678383b76/ppca-0.0.4-py3-none-any.whl
    Requirement already satisfied: pandas>=0.18.0 in /usr/local/lib/python3.7/dist-packages (from hypertools) (1.1.5)
    Collecting umap-learn>=0.4.6
      Downloading https://files.pythonhosted.org/packages/75/69/85e7f950bb75792ad5d666d86c5f3e62eedbb942848e7e3126513af9999c/umap-learn-0.5.1.tar.gz (80kB)
         |████████████████████████████████| 81kB 4.9MB/s 
    Requirement already satisfied: cycler>=0.10 in /usr/local/lib/python3.7/dist-packages (from matplotlib>=1.5.1->hypertools) (0.10.0)
    Requirement already satisfied: python-dateutil>=2.1 in /usr/local/lib/python3.7/dist-packages (from matplotlib>=1.5.1->hypertools) (2.8.1)
    Requirement already satisfied: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=2.0.1 in /usr/local/lib/python3.7/dist-packages (from matplotlib>=1.5.1->hypertools) (2.4.7)
    Requirement already satisfied: kiwisolver>=1.0.1 in /usr/local/lib/python3.7/dist-packages (from matplotlib>=1.5.1->hypertools) (1.3.1)
    Requirement already satisfied: tables in /usr/local/lib/python3.7/dist-packages (from deepdish->hypertools) (3.4.4)
    Requirement already satisfied: joblib>=0.11 in /usr/local/lib/python3.7/dist-packages (from scikit-learn!=0.22,<0.24,>=0.19.1->hypertools) (1.0.1)
    Requirement already satisfied: chardet<4,>=3.0.2 in /usr/local/lib/python3.7/dist-packages (from requests->hypertools) (3.0.4)
    Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.7/dist-packages (from requests->hypertools) (2021.5.30)
    Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /usr/local/lib/python3.7/dist-packages (from requests->hypertools) (1.24.3)
    Requirement already satisfied: idna<3,>=2.5 in /usr/local/lib/python3.7/dist-packages (from requests->hypertools) (2.10)
    Requirement already satisfied: pytz>=2017.2 in /usr/local/lib/python3.7/dist-packages (from pandas>=0.18.0->hypertools) (2018.9)
    Requirement already satisfied: numba>=0.49 in /usr/local/lib/python3.7/dist-packages (from umap-learn>=0.4.6->hypertools) (0.51.2)
    Collecting pynndescent>=0.5
      Downloading https://files.pythonhosted.org/packages/b1/8d/44bf1c9e69dd9bf0697a3b9375b0729942525c0eee7b7859f563439d676a/pynndescent-0.5.4.tar.gz (1.1MB)
         |████████████████████████████████| 1.1MB 10.1MB/s 
    Requirement already satisfied: numexpr>=2.5.2 in /usr/local/lib/python3.7/dist-packages (from tables->deepdish->hypertools) (2.7.3)
    Requirement already satisfied: llvmlite<0.35,>=0.34.0.dev0 in /usr/local/lib/python3.7/dist-packages (from numba>=0.49->umap-learn>=0.4.6->hypertools) (0.34.0)
    Requirement already satisfied: setuptools in /usr/local/lib/python3.7/dist-packages (from numba>=0.49->umap-learn>=0.4.6->hypertools) (57.0.0)
    Building wheels for collected packages: umap-learn, pynndescent
      Building wheel for umap-learn (setup.py) ... done
      Created wheel for umap-learn: filename=umap_learn-0.5.1-cp37-none-any.whl size=76569 sha256=9724e07d61b5a5b9f10255907b8612fd2a445bbda2812801a47378be1026ae83
      Stored in directory: /root/.cache/pip/wheels/ad/df/d5/a3691296ff779f25cd1cf415a3af954b987fb53111e3392cf4
      Building wheel for pynndescent (setup.py) ... done
      Created wheel for pynndescent: filename=pynndescent-0.5.4-cp37-none-any.whl size=52374 sha256=5b61db3df31eb793b253bd1a7fb0330841623781450a9a95ceb0ae682331206f
      Stored in directory: /root/.cache/pip/wheels/42/4b/8c/f6f119c67cf6583bb192431fa8f7278cf95e5b943055077d94
    Successfully built umap-learn pynndescent
    Installing collected packages: deepdish, PPCA, pynndescent, umap-learn, hypertools
    Successfully installed PPCA-0.0.4 deepdish-0.3.6 hypertools-0.7.0 pynndescent-0.5.4 umap-learn-0.5.1""")
    expected = ['PPCA-0.0.4 deepdish-0.3.6 hypertools-0.7.0 pynndescent-0.5.4 umap-learn-0.5.1']
    result = pip_installed_pkgs_regex.findall(stdout)
    assert result == expected, (
        f"Expected: {expected}\nFound: {result}"
    )

In [ ]:
def test_pip_installed_regex_none():
    """
    shouldn't match any package names when the package was already 
    installed. stdout used for test comes from running 
    '!pip install requests' in a Colab notebook.
    """
    stdout = dedent("""\
    Requirement already satisfied: requests in /usr/local/lib/python3.7/dist-packages (2.23.0)
    Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.7/dist-packages (from requests) (2021.5.30)
    Requirement already satisfied: idna<3,>=2.5 in /usr/local/lib/python3.7/dist-packages (from requests) (2.10)
    Requirement already satisfied: chardet<4,>=3.0.2 in /usr/local/lib/python3.7/dist-packages (from requests) (3.0.4)
    Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /usr/local/lib/python3.7/dist-packages (from requests) (1.24.3)""")
    expected = []
    result = pip_installed_pkgs_regex.findall(stdout)
    assert result == expected, (
        f"Expected: {expected}\nFound: {result}"
    )

In [ ]:
def test_smuggle_regex_ignores_none():
    """should not match simple, unrelated line"""
    line = 'def foo(bar, baz=qux):'
    match = smuggle_statement_regex.match(line)
    assert match is None, f"matched: '{match.group()}'"

In [ ]:
def test_smuggle_regex_ignores_decoy():
    """should not match line containing "smuggle" in unrelated context"""
    line = 'def smuggle_something(foo):'
    match = smuggle_statement_regex.match(line)
    assert match is None, f"matched: '{match.group()}'"

In [ ]:
def test_smuggle_regex_ignores_comment():
    """should not match commented-out smuggle statement"""
    line = '# smuggle foo as bar'
    match = smuggle_statement_regex.match(line)
    assert match is None, f"matched: '{match.group()}'"

In [ ]:
def test_smuggle_regex_ignores_non_leading():
    """should not match smuggle statements not at beginning of line"""
    line = 'foo() and bar() and baz() smuggle qux()'
    match = smuggle_statement_regex.match(line)
    assert match is None, f"matched: '{match.group()}'"

In [ ]:
def test_smuggle_regex_basic():
    """simplest use case"""
    line = 'smuggle foo'
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_regex_alias():
    """simplest use case, plus alias"""
    line = 'smuggle foo as bar'
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_regex_basic_onion():
    """simplest use case, plus onion comment"""
    line = 'smuggle foo    # pip: foo==0.0.1'
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': '# pip: foo==0.0.1',
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_regex_alias_onion():
    """simplest use case, plus alias, plus onion comment"""
    line = 'smuggle foo as bar    # pip: foo==0.0.1'
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': '# pip: foo==0.0.1',
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_regex_conda_onion():
    """same as above, but with conda specified as the installer"""
    line = 'smuggle foo as bar    # conda: foo==0.0.1'
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': '# conda: foo==0.0.1',
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_regex_alias_ignores_non_onion():
    """same as above, but with a *non-onion* inline comment"""
    line = 'smuggle foo as bar    # load the module that does the thing'
    expected_groupdict = {
        'FULL_CMD': 'smuggle foo as bar',
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_regex_alias_ignores_decoy_onion():
    """same as above, but with a *just barely non-onion* comment"""
    line = 'smuggle foo as bar    # pip; "a [redacted] package manager than conda"'
    expected_groupdict = {
        'FULL_CMD': 'smuggle foo as bar',
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_regex_qualname():
    """should match qualified names of packages/modules"""
    line = 'smuggle foo.bar as baz'
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_regex_indented():
    """
    should match smuggle statements inside try/except blocks, loops, 
    if blocks, function definitions, etc.
    """
    line = '    smuggle foo as bar    # pip: foo==0.0.1'
    expected_groupdict = {
        'FULL_CMD': line.strip(),
        'SEMICOLON_SEP': None,
        'ONION': '# pip: foo==0.0.1',
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_regex_multiple_pkgs():
    """
    should match line with multiple packages (with and without aliases, 
    with and without qualnames) smuggled using a single 'smuggle' 
    statement
    """
    line = 'smuggle foo as bar, baz as qux, spam.ham, eggs'
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_regex_multiple_pkgs_onion():
    """same as above, with onion comment"""
    line = 'smuggle foo as bar, baz as qux, spam.ham, eggs    # pip: foo==0.0.1'
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': '# pip: foo==0.0.1',
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_regex_multiline_explicit():
    """
    same as above, but split across lines with explicit 
    (backslash-based) line continuations
    """
    line = """smuggle foo as bar, \
              baz as qux, \
              spam.ham, \
              eggs    # pip: foo==0.0.1"""
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': '# pip: foo==0.0.1',
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_regex_whitespace_insensitive():
    """
    same as above, with inconsistent but syntactically valid whitespace, 
    including explicit (backslash-based) line continuations
    """
    line = """smuggle               foo     as    bar    \
,baz as   qux  , \
          spam  .  ham    ,    eggs                #   pip   :       foo==0.0.1    """
    expected_groupdict = {
        'FULL_CMD': line.strip(),
        'SEMICOLON_SEP': None,
        'ONION': '#   pip   :       foo==0.0.1',
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_regex_from():
    """simplest use case from second syntactic category"""
    line = 'from foo smuggle bar'
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_regex_from_alias():
    """same as above, with alias"""
    line = 'from foo smuggle bar as baz'
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_regex_from_onion():
    """same as above, with onion comment"""
    line = 'from foo smuggle bar    # pip: foo==0.0.1'
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': '# pip: foo==0.0.1'
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_regex_from_alias_onion():
    """same as above, with alias, with onion comment"""
    line = 'from foo smuggle bar as baz    # pip: foo==0.0.1'
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': '# pip: foo==0.0.1'
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_regex_from_qualname():
    """adds qualname to above setup"""
    line = 'from foo.bar.baz smuggle qux as quux    # pip: foo==0.0.1'
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': '# pip: foo==0.0.1'
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_regex_from_multiple_names_onion():
    """
    should match line with multiple names (with and without aliases, 
    with and without qualnames) being smuggled from the same package 
    using a single 'smuggle' statement, followed by an onion comment
    """
    line = 'from foo.bar smuggle baz, qux as quux, spam, ham as eggs    # pip: foo==0.0.1'
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': '# pip: foo==0.0.1'
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_from_parentheses():
    """
    same as above, but names are enclosed in parentheses (even though 
    they are not split across multiple lines). Also tests trailing comma 
    inside parentheses, which is syntactically valid for import 
    statements.
    """
    line = 'from foo.bar smuggle (baz, qux as quux, spam, ham as eggs,)    # pip: foo==0.0.1'
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': '(',
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': ')',
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': '# pip: foo==0.0.1'
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_from_multiline_explicit():
    """
    similar to above, but split across multiple lines with explicit 
    (backslash-based) line continuations
    """
    line = """from foo.bar smuggle baz, \
                                   qux as quux, \
                                   spam, \
                                   ham as eggs    # pip: foo==0.0.1"""
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': '# pip: foo==0.0.1'
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_from_multiline_implicit_1_onion_1():
    """
    similar to above, but 'smuggle' statement is split across multiple 
    lines with implicit (parentheses-based) line continuation. First of 
    multiple variations on multiline syntax.
    
    Notes:
    - implicit_1: first/last smuggled names occupy same lines as 
                  opening/closing parentheses
    - onion_1: onion comment appears on first line
    """
    line = """from foo.bar smuggle (baz,    # pip: foo==0.0.1
                                    qux as quux,
                                    spam, 
                                    ham as eggs)"""
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': '(',
        'FROM_ONION_1': '# pip: foo==0.0.1',
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_from_multiline_implicit_1_onion_2():
    """
    Variation on above.
    
    Notes:
    - implicit_1: first/last smuggled names occupy same lines as 
                  opening/closing parentheses
    - onion_2: onion comment appears on last line
    """
    line = """from foo.bar smuggle (baz,
                                    qux as quux,
                                    spam, 
                                    ham as eggs)    # pip: foo==0.0.1"""
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': '(',
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': '# pip: foo==0.0.1'
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_from_multiline_implicit_1_onion_3():
    """
    Variation on above.
    
    Notes:
    - implicit_1: first/last smuggled names occupy same lines as 
                  opening/closing parentheses
    - onion_3: onion comment appears on line with name, but not first or 
               last, so not parsed
    """
    line = """from foo.bar smuggle (baz,
                                    qux as quux,    # pip: foo==0.0.1
                                    spam, 
                                    ham as eggs)"""
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': '(',
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_from_multiline_implicit_1_onion_4():
    """
    Variation on above.
    
    Notes:
    - implicit_1: first/last smuggled names occupy same lines as 
                  opening/closing parentheses
    - onion_4: onion comment appears on separate line, but not first or 
               last, so not parsed
    """
    line = """from foo.bar smuggle (baz,
                                    # pip: foo==0.0.1
                                    qux as quux,
                                    spam, 
                                    ham as eggs)"""
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': '(',
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_from_multiline_implicit_1_onion_5():
    """
    Variation on above.
    
    Notes:
    - implicit_1: first/last smuggled names occupy same lines as 
                  opening/closing parentheses
    - onion_5: onion comment appears on last line, with non-onion 
               comments on various lines above it
    """
    line = """from foo.bar smuggle (baz,    # unrelated first line comment
                                    qux as quux,    # unrelated middle line comment
                                    spam, 
                                    # unrelated separate line comment
                                    ham as eggs)    # pip: foo==0.0.1"""
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': '(',
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': '# pip: foo==0.0.1'
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_from_multiline_implicit_2_onion_1():
    """
    Variation on above.
    
    Notes:
    - implicit_2: first/last smuggled names occupy different lines from
                  opening/closing parentheses
    - onion_1: onion comment appears on first line
    """
    line = """from foo.bar smuggle (    # pip: foo==0.0.1
                  baz,
                  qux as quux,
                  spam, 
                  ham as eggs
              )"""
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': '(',
        'FROM_ONION_1': '# pip: foo==0.0.1',
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_from_multiline_implicit_2_onion_2():
    """
    Variation on above.
    
    Notes:
    - implicit_2: first/last smuggled names occupy different lines from
                  opening/closing parentheses
    - onion_2: onion comment appears on last line
    """
    line = """from foo.bar smuggle (
                  baz,
                  qux as quux,
                  spam, 
                  ham as eggs
              )    # pip: foo==0.0.1"""
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': '(',
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': '# pip: foo==0.0.1'
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_from_multiline_implicit_2_onion_3():
    """
    Variation on above.
    
    Notes:
    - implicit_2: first/last smuggled names occupy different lines from
                  opening/closing parentheses
    - onion_3: onion comment appears on line with name, but not first or 
               last, so not parsed
    """
    line = """from foo.bar smuggle (
                  baz, 
                  qux as quux,
                  spam,    # pip: foo==0.0.1
                  ham as eggs
              )"""
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': '(',
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_from_multiline_implicit_2_onion_4():
    """
    Variation on above.
    
    Notes:
    - implicit_2: first/last smuggled names occupy different lines from
                  opening/closing parentheses
    - onion_4: onion comment appears on separate line, but not first or 
               last, so not parsed
    """
    line = """from foo.bar smuggle (
                  baz, 
                  qux as quux,
                  # pip: foo==0.0.1
                  spam,
                  ham as eggs
              )"""
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': '(',
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_from_multiline_implicit_2_onion_5():
    """
    Variation on above.
    
    Notes:
    - implicit_2: first/last smuggled names occupy different lines from
                  opening/closing parentheses
    - onion_5: onion comment appears on last line, with non-onion 
               comments on various lines above it
    """
    line = """from foo.bar smuggle (    # unrelated first line comment
                  baz,
                  qux as quux,    # unrelated middle line comment
                  spam, 
                  # unrelated separate line comment
                  ham as eggs
              )    # pip: foo==0.0.1"""
    expected_groupdict = {
        'FULL_CMD': line,
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': '(',
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': '# pip: foo==0.0.1'
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_multi_semicolon():
    """
    should handle multiple semicolon-separated 'smuggle' statements on a 
    single line. Match should include first statement only, but note 
    that there are other statements on the same line via the 
    'SEMICOLON_SEP' group matching. Should *not* include onion comment, 
    since that applies to last statement on line. Also incorporates 
    multiple elements from prior tests.
    """
    line = """smuggle foo; from bar smuggle baz; \
              smuggle qux as quux; \
              from spam smuggle ham as eggs    # pip: spam-pkg==0.0.1"""
    expected_groupdict = {
        'FULL_CMD': 'smuggle foo',
        'SEMICOLON_SEP': '',
        'ONION': None,
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': None,
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
def test_smuggle_from_multi_semicolon():
    """
    Same as above, but the 'FROM_SEMICOLON_SEP' group should match 
    instead of 'SEMICOLON_SEP', since the first statement uses 'from'
    """
    line = """from foo smuggle bar; smuggle baz; \
              smuggle qux as quux; \
              from spam smuggle ham as eggs    # pip: spam-pkg==0.0.1"""
    expected_groupdict = {
        'FULL_CMD': 'from foo smuggle bar',
        'SEMICOLON_SEP': None,
        'ONION': None,
        'OPEN_PARENS': None,
        'FROM_ONION_1': None,
        'CLOSE_PARENS_FIRSTLINE': None,
        'FROM_SEMICOLON_SEP': '',
        'FROM_ONION': None
    }
    match = smuggle_statement_regex.match(line)
    assert match is not None, f"failed to match: '{line}'"
    
    result_groupdict = match.groupdict()
    assert result_groupdict == expected_groupdict, (
        "bad values in match.groupdict(). Expected:\n"
        f"{pformat(expected_groupdict)}\nFound:\n{pformat(result_groupdict)}"
    )

In [ ]:
run_tests()